# Code Constructions

Build HGP, QLP, and BPC codes and verify their CSS logicals.


In [1]:
import numpy as np
import stim

from quits.circuit import check_overlapping_CX
from quits import ErrorModel, CircuitBuildOptions, get_cardinal_circuit
from quits.decoder import (
    sliding_window_bposd_circuit_mem,
    sliding_window_bposd_phenom_mem,
    sliding_window_bplsd_phenom_mem,
)
from quits.qldpc_code import BpcCode, HgpCode, QlpCode
from quits.simulation import get_stim_mem_result


### Hypergraph product (HGP) code parameters

Reference: Tillich & Zemor, arXiv:0903.0566

- `h`: parity check matrix of the classical LDPC code (loaded from `../parity_check_matrices`)
- `HgpCode(h, h)`: constructs the HGP code from two classical parity check matrices
- `seed`: used by `build_circuit` to assign edge directions and colors

The parity check matrix `h` can be found from `quits.ldpc_utility.generate_ldpc`, following the method in [A. Grospellier, *Constant time decoding of quantum expander codes and application to fault-tolerant quantum computation* (2019)].

In [2]:
# Build a small HGP code
h = np.loadtxt(
    "../parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt",
    dtype=int,
)
code = HgpCode(h, h, verbose=True)
code.build_circuit(seed=22)
# For this particular parity check matrix, seed=22 is the one that gives entangling depth 8 (other seeds may give entangling depth 12).
code.verify_css_logicals()


HgpCode: using canonical logical codewords.


{'css_condition': True,
 'lz_commutes_with_X': True,
 'lx_commutes_with_Z': True,
 'rank_hz': 108,
 'rank_hx': 108,
 'rank_lz': 9,
 'rank_lx': 9,
 'k_expected': 9,
 'lz_independent_mod_Z_stabilizers': True,
 'lx_independent_mod_X_stabilizers': True,
 'rank_hz_plus_lz': 117,
 'rank_hx_plus_lx': 117,
 'dim_ker_hz': 117,
 'dim_ker_hx': 117,
 'hx_plus_lx_spans_ker_hz': True,
 'hz_plus_lz_spans_ker_hx': True,
 'same_logicals_ZX_anticommute': True,
 'different_logicals_ZX_commute': True,
 'all_tests_passed': True}

**Canonical logical codewords.** For the HGP code, logical operators are written in the canonical form (see arXiv:2204.10812):
$$L^z_{ik_1+j} = e_i \otimes L^1_j, \quad L^x_{ik_1+j} = L^2_i \otimes e_j$$
where $L^1_j$ ($L^2_i$) is the $j$-th ($i$-th) logical operator of the 1st (2nd) classical code.


In [3]:
# Print canonical logical codewords for the HGP code
for i in range(code.lz.shape[0]):
    print(f'Z{i} :', np.where(code.lz[i, :] == 1)[0], f', X{i} :', np.where(code.lx[i, :] == 1)[0])

Z0 : [111 113 114 115 116 117] , X0 : [ 45  69  81  93 105 117]
Z1 : [108 109 113 114 116 118] , X1 : [ 46  70  82  94 106 118]
Z2 : [110 112 113 114 116 119] , X2 : [ 47  71  83  95 107 119]
Z3 : [123 125 126 127 128 129] , X3 : [  9  21  69  81 105 129]
Z4 : [120 121 125 126 128 130] , X4 : [ 10  22  70  82 106 130]
Z5 : [122 124 125 126 128 131] , X5 : [ 11  23  71  83 107 131]
Z6 : [135 137 138 139 140 141] , X6 : [ 33  57  69  81 105 141]
Z7 : [132 133 137 138 140 142] , X7 : [ 34  58  70  82 106 142]
Z8 : [134 136 137 138 140 143] , X8 : [ 35  59  71  83 107 143]


### Quasi-cyclic Lifted Product (QLP) code parameters

Reference: Q. Xu et al., arXiv:2308.08648 (see Table below)

- `b`: base matrix of monomial powers for the polynomial entries
- `lift_size`: circulant size used to lift each monomial entry
- `QlpCode(b, b, lift_size)`: builds a QLP code from two base matrices

Example parameters used in arXiv:2308.08648 are shown below.

| lift_size | [[n, k, d]] | b |
|:---:|:---:|:---:|
| 16 | [[544, 80, $\leq$ 12]] | $\begin{bmatrix} 0 & 0 & 0 & 0 & 0 \\ 0 & 2 & 4 & 7 & 11 \\ 0 & 3 & 10 & 14 & 15 \end{bmatrix}$ |
| 21 | [[714, 100, $\leq$ 16]] | $\begin{bmatrix} 0 & 0 & 0 & 0 & 0 \\ 0 & 4 & 5 & 7 & 17 \\ 0 & 14 & 18 & 12 & 11 \end{bmatrix}$ |
| 30 | [[1020, 136, $\leq$ 20]] | $\begin{bmatrix} 0 & 0 & 0 & 0 & 0 \\ 0 & 2 & 14 & 24 & 25 \\ 0 & 16 & 11 & 14 & 13 \end{bmatrix}$ |
| 42 | [[1428, 184, $\leq$ 24]] | $\begin{bmatrix} 0 & 0 & 0 & 0 & 0 \\ 0 & 6 & 7 & 9 & 30 \\ 0 & 40 & 15 & 31 & 35 \end{bmatrix}$ |



In [4]:
# Build a small QLP code
lift_size = 16
b = np.array(
    [
        [0, 0, 0, 0, 0],
        [0, 2, 4, 7, 11],
        [0, 3, 10, 14, 15],
    ]
)
code = QlpCode(b, b, lift_size)
code.build_circuit(seed=1)
code.verify_css_logicals()


{'css_condition': True,
 'lz_commutes_with_X': True,
 'lx_commutes_with_Z': True,
 'rank_hz': 232,
 'rank_hx': 232,
 'rank_lz': 80,
 'rank_lx': 80,
 'k_expected': 80,
 'lz_independent_mod_Z_stabilizers': True,
 'lx_independent_mod_X_stabilizers': True,
 'rank_hz_plus_lz': 312,
 'rank_hx_plus_lx': 312,
 'dim_ker_hz': 312,
 'dim_ker_hx': 312,
 'hx_plus_lx_spans_ker_hz': True,
 'hz_plus_lz_spans_ker_hx': True,
 'same_logicals_ZX_anticommute': True,
 'different_logicals_ZX_commute': True,
 'all_tests_passed': True}

### Balanced product cyclic (BPC) code parameters

Reference: R. Tiew et al., arXiv:2411.03302

- `p1`, `p2`: polynomial powers defining the two circulant blocks
- `lift_size`: circulant size of each block
- `factor`: size of the cyclic subgroup factored out

To match arXiv:2411.03302, use `p2` as the transpose convention: each entry is `lift_size` minus the powers listed in the paper, as shown in the table below. Use **lift_size = 3q** and **factor = 3**.

| lift_size (=3q) | [[n, k, d]] | 2q | p1 | p2 |
|:---:|:---:|:---:|:---:|:---:|
| 6 | [[36, 8, 4]] | 4 | [0, 1, 2] | [0, 4, 5] |
| 9 | [[54, 8, 4]] | 6 | [0, 1, 2] | [0, 7, 8] |
| 12 | [[72, 8, 8]] | 8 | [0, 1, 5] | [0, 3, 11] |
| 15 | [[90, 8, 10]] | 10 | [0, 1, 5] | [0, 8, 13] |
| 18 | [[108, 8, 8]] | 12 | [0, 1, 5] | [0, 16, 17] |
| 21 | [[126, 8, 10]] | 14 | [0, 1, 5] | [0, 14, 19] |
| 24 | [[144, 8, 12]] | 16 | [0, 1, 5] | [0, 13, 23] |
| 27 | [[162, 8, 12]] | 18 | [0, 1, 5] | [0, 16, 26] |
| 30 | [[180, 8, 16]] | 20 | [0, 1, 5] | [0, 13, 29] |


In [5]:
# Build a small BPC code
lift_size, factor = 15, 3
p1 = [0, 1, 5]
p2 = [0, 8, 13]
# p2 uses lift_size minus values compared to arXiv:2411.03302 (transpose convention).
code = BpcCode(p1, p2, lift_size, factor, canonical_basis='Z', verbose=True)
code.build_circuit(seed=1)
code.verify_css_logicals()


BpcCode: using canonical logical codewords (q is odd).


{'css_condition': True,
 'lz_commutes_with_X': True,
 'lx_commutes_with_Z': True,
 'rank_hz': 41,
 'rank_hx': 41,
 'rank_lz': 8,
 'rank_lx': 8,
 'k_expected': 8,
 'lz_independent_mod_Z_stabilizers': True,
 'lx_independent_mod_X_stabilizers': True,
 'rank_hz_plus_lz': 49,
 'rank_hx_plus_lx': 49,
 'dim_ker_hz': 49,
 'dim_ker_hx': 49,
 'hx_plus_lx_spans_ker_hz': True,
 'hz_plus_lz_spans_ker_hx': True,
 'same_logicals_ZX_anticommute': True,
 'different_logicals_ZX_commute': True,
 'all_tests_passed': True}

**Canonical logical codewords.** For the BPC code with odd $q$, logical operators are written in the canonical form, such that codewords of canonical\_basis are of weight $2q$. This is modified from Eq. 30 of arXiv:2411.03302 so that `same_logicals\_ZX\_anticommute' and 'different\_logicals\_ZX\_commute' are True. 


In [6]:
# Print canonical logical codewords for the HGP code
for i in range(code.lz.shape[0]):
    print(f'Z{i} :', np.where(code.lz[i, :] == 1)[0], f', X{i} :', np.where(code.lx[i, :] == 1)[0])

Z0 : [45 46 48 49 51 52 54 55 57 58] , X0 : [45 48 51 54 57 75 78 81 84 87]
Z1 : [46 47 49 50 52 53 55 56 58 59] , X1 : [45 46 48 49 51 52 54 55 57 58 75 76 78 79 81 82 84 85 87 88]
Z2 : [60 61 63 64 66 67 69 70 72 73] , X2 : [60 63 66 69 72 75 78 81 84 87]
Z3 : [61 62 64 65 67 68 70 71 73 74] , X3 : [60 61 63 64 66 67 69 70 72 73 75 76 78 79 81 82 84 85 87 88]
Z4 : [ 0  3  6  9 12 15 18 21 24 27] , X4 : [ 0  2  3  5  6  8  9 11 12 14]
Z5 : [ 1  4  7 10 13 16 19 22 25 28] , X5 : [ 1  2  4  5  7  8 10 11 13 14]
Z6 : [15 18 21 24 27 30 33 36 39 42] , X6 : [ 0  2  3  5  6  8  9 11 12 14 15 17 18 20 21 23 24 26 27 29]
Z7 : [16 19 22 25 28 31 34 37 40 43] , X7 : [ 1  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29]
